###  4.1 层次化索引

#### 1. Series的层次化索引
1. 层次化索引表示 , 一个轴上, 有2个以上的索引级别  
 创建一个多级索引结构的Series, 由多个列表/数组构成的列表即多级索引  
 索引之间的间隔, 表示直接使用上面的标签
2. 通过最外层索引选取数据, 若原先是Series,则返回Series. 若原先是DataFrame,则返回DataFrame  
 返回的数据是索引减少1级的数据结构 
3. Series内层索引选取, 使用obj[outerIdx,innerIdx]

In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

# 多级索引结构的Series
obj = Series(np.random.randn(9),
            index = [list('aaabbccdd'),list('123131223')])
obj


a  1    1.459606
   2   -0.548454
   3    0.112091
b  1    1.399476
   3   -0.001406
c  1    0.295772
   2    0.343392
d  2    0.339913
   3   -1.222963
dtype: float64

In [2]:
# 多集索引
print obj.index
print "================[1]=============="
# 返回Series
print obj['b']
print "================[2]=============="
# 选取多个外层索引的数据
print obj[['b','c']]
print "================[3]=============="
# 选取内层索引的数据
obj.loc[:,'2']

MultiIndex(levels=[[u'a', u'b', u'c', u'd'], [u'1', u'2', u'3']],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])
================[1]==============
1    1.399476
3   -0.001406
dtype: float64
================[2]==============
b  1    1.399476
   3   -0.001406
c  1    0.295772
   2    0.343392
dtype: float64
================[3]==============


a   -0.548454
c    0.343392
d    0.339913
dtype: float64

#### 2. stack与unstack
1. Series.unstack() : 解决堆叠, 将Series的内层索引, 拓宽成columnIdx. 返回DataFrame
2. DataFrame.stack() : 堆叠 把columnIdx拉长成内层索引, 返回Series

In [3]:
df = obj.unstack()
df

,1,2,3
a,1.459606,-0.548454,0.112091
b,1.399476,NaN,-0.001406
c,0.295772,0.343392,NaN
d,NaN,0.339913,-1.222963


In [4]:
df.stack()

a  1    1.459606
   2   -0.548454
   3    0.112091
b  1    1.399476
   3   -0.001406
c  1    0.295772
   2    0.343392
d  2    0.339913
   3   -1.222963
dtype: float64

#### 3. DataFrame的层次化索引
1. DataFrame的每个轴上, 都可以是层次化索引. 即index和columns都能是列表/数组组成的列表
2. 可以对每层的索引起名字:  
  1. DataFrame.index.names=[str] : 对rowIdx的每层起名字
  2. DataFrame.columns.names = [str] , 对columnIdx每层起名字
  
 3. 在多层columnIdx上选取DataFrame  
  DataFrame['outIdx','innerIdx']或者精确到内层索引: DataFrame['outIdx','innerIdx']

In [5]:
# index和column均为多集索引的DataFrame
df = DataFrame(np.arange(12).reshape(4,3),
              index = [list('aaba'),list('1212')],
              columns=[['Ohio', 'Ohio', 'Colorado'],['Green', 'Red', 'Green']])
df

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
a 2     9  10       11

In [6]:
df.index.names=['key1','key2']
df.columns.names=['state','color']

df

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
a    2        9  10       11

In [7]:
df['Ohio','Green']

key1  key2
a     1       0
      2       3
b     1       6
a     2       9
Name: (Ohio, Green), dtype: int64

#### 4. DataFrame的索引层级重排与索引排序
1. 交换DataFrame的内外层行索引顺序 :  
 DataFrame.swaplevel('rowIdx1','rowIdx2',axis=0) : 默认axis=0, 交换行索引
2. DataFrame.Sort_index(level=[n,n]) : DataFrame分别在某个级别上的索引进行排序

In [8]:
print df.index.name
# df.swaplevel('key1', 'key2',axis=0)
df.swaplevel(0,1,axis=0)

None


state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    a        9  10       11

In [9]:
df.swaplevel('key1','key2',axis=0).sort_index(level=[0,1])
# df.swaplevel(0,1,axis=0).sort_index(level=[0,1])

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     a        9  10       11

#### 5. 多级索引下的求和
1. 可以在多集索引的某个level上求和.  
 DataFrame.sum(level='',axis=n)

In [10]:
df.sum(level='color',axis=1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
a    2        20   10

#### 6. DataFrame的列转换成行索引
1. DataFrame.set_index([columnIdx1,columnIdx2]) :  
 将1个/多个columnIdx作为行索引. 并将这两个column删除和原先的索引  
 若要保留这2个column, 指定参数frop=False  
 若要保留原先的index, 指定参数append=True
 
2. DataFrame.reset_index() : 将所有层次的行索引变成columnIdx

In [11]:
df = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'], 'd': [0, 1, 2, 0, 1, 2, 3]},
                 index=list('zyxwvut'))
df

,a,b,c,d
z,0,7,one,0
y,1,6,one,1
x,2,5,one,2
w,3,4,two,0
v,4,3,two,1
u,5,2,two,2
t,6,1,two,3


In [12]:
df2 = df.set_index(['c','d'])
print df2.index
df2

MultiIndex(levels=[[u'one', u'two'], [0, 1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1, 1], [0, 1, 2, 0, 1, 2, 3]],
           names=[u'c', u'd'])


a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [13]:
df3 = df.set_index(['c','d'],append=True)
print df3.index
df3

MultiIndex(levels=[[u't', u'u', u'v', u'w', u'x', u'y', u'z'], [u'one', u'two'], [0, 1, 2, 3]],
           labels=[[6, 5, 4, 3, 2, 1, 0], [0, 0, 0, 1, 1, 1, 1], [0, 1, 2, 0, 1, 2, 3]],
           names=[None, u'c', u'd'])


,,,a,b
,c,d,,
z,one,0,0,7
y,one,1,1,6
x,one,2,2,5
w,two,0,3,4
v,two,1,4,3
u,two,2,5,2
t,two,3,6,1


In [14]:
df3.reset_index()

,level_0,c,d,a,b
0,z,one,0,0,7
1,y,one,1,1,6
2,x,one,2,2,5
3,w,two,0,3,4
4,v,two,1,4,3
5,u,two,2,5,2
6,t,two,3,6,1


### 4.2 合并数据集

#### 1. DataFrame的join操作 : pandas.merge(df1,df2)
1. merge(df1,df2) :   
 默认使用重合的columnIdx作为键进行连接. 但最好门明确制定join的外键(参数on='columnIdx')  
2. 若join的两个DataFrame的columnIdx名称不一致,   
 则可指定参数left_on='left_columnIdx', right_on='right_columnIdx'  
 merge默认进行inner join, 2个外键对应不上的值会被抛弃
3.  参数how='outer/left/right' :  
 分别指定连接方式为"全连接","左链接", "右连接"  
4. 有时2个DataFrame啊join后, 存在重叠的columnIdx, 为了区分这些columnIdx来自于哪个DataFrame, 使用参数suffixes=('left_suffix','right_suffix')进行区分

In [15]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': np.arange(7)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [16]:
df2 = DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [17]:
# columnIdx一致的外键
pd.merge(df1,df2,on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [18]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],  'data2': range(3)})

In [19]:
df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [20]:
df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [21]:
# 2个DataFrame, join的column name不同的情况
# merge默认inner join, 发现lkey取值为c,rkey取值为d的2中数据已被抛弃
pd.merge(df3,df4,left_on='lkey',right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [22]:
# 全连接 (保留对应不上的2个外键)- 会出现NaN值
pd.merge(df3,df4,left_on='lkey',right_on='rkey',how='outer')

,data1,lkey,data2,rkey
0,0.0,b,1.0,b
1,1.0,b,1.0,b
2,6.0,b,1.0,b
3,2.0,a,0.0,a
4,4.0,a,0.0,a
5,5.0,a,0.0,a
6,3.0,c,NaN,NaN
7,NaN,NaN,2.0,d


In [23]:
pd.merge(df3,df3,left_on="lkey",right_on="lkey",how="inner",suffixes=("_left","_right"))

,data1_left,lkey,data1_right
0,0,b,0
1,0,b,1
2,0,b,6
3,1,b,0
4,1,b,1
5,1,b,6
6,6,b,0
7,6,b,1
8,6,b,6
9,2,a,2


#### 2.  多列clumnId作为merge的外键
1. 多列合并, 只有当这多个columnIdx的取值全都相同时, join才能成功
2. 多列合并, 可理解为 : 多个键形成一系列元祖, 并将其当做单个连接键进行join (事实上不是这么回事)

In [24]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],'key2': ['one', 'two', 'one'], 'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],'key2': ['one', 'one', 'one', 'two'],'rval': [4, 5, 6, 7]})
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


#### 3. 索引上的join
1. 参数left_index=True/right_index=True指定左表或右表使用index作为外键.  
2. 左表的index可以和右表的columnIdx作连接, 使用(left_index=True, right_on="columnIdx")
3. 若两个表都使用index进行join, 则使用left_index=True,right_index=True   
4. 多级索引的情况, 假如左表的索引为多级的, left_index=True, 则指明右表的columnIdx为多个 right_on=['columnIdx1',''columnIdx2]

In [25]:
# 左表的列和右表的index进行join
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
print left1
print "==============[1]============="
print right1
print "==============[2]============="
pd.merge(left1,right1,left_on="key",right_index=True, how="left")

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
==============[1]=============
   group_val
a        3.5
b        7.0
==============[2]=============


,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [26]:
# 多级索引和多个column进行join
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio','Nevada', 'Nevada'],
                                    'key2': [2000, 2001, 2002, 2001, 2002],
                                    'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio','Ohio', 'Ohio'],
                                      [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
print lefth
print "==============[1]============="
print righth
print "==============[2]============="
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='outer')

   data    key1  key2
0   0.0    Ohio  2000
1   1.0    Ohio  2001
2   2.0    Ohio  2002
3   3.0  Nevada  2001
4   4.0  Nevada  2002
==============[1]=============
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11
==============[2]=============


,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


### 4.3. Series的轴向连接
轴向连接, 指将多个数据结构对象在某个轴的方向上连接起来

#### 1. Series的轴连接
1. Series默认在行的方向上轴连接
2. 当轴连接的几个Series之间, 存在堆叠的索引时  
  1. 若在行上连接, 则直接连接, 即使存在索引会在新Series中重复出现
  2. 若在列上进行连接, 则意思为根据这几个Series的index进行join, 因此通过指定参数join='inner/outter'来实现join拼接的逻辑  
       此时, 可通过参数join_axes=[[list('indexes')]]指定inner/outter连接后出现的索引.
3. 为了能表示出连接后的Series, 数据来自原先的哪个Series. 可以通过层次化索引进行. 指定参数keys=['s1','s2',...]  
  1. 若在行上连接, 则key成为数据索引的上层行索引
  2. 若在列上连接, 则key成为DataFrame的columnIdx

In [27]:
# 1. 3个美誉堆叠索引的Series轴连接
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
print s1
print s2
print s3
print "==============[1]============="
print pd.concat([s1,s2,s3], axis=0)  # 形成高的Series
print "==============[2]============="
print pd.concat([s1,s2,s3], axis=1)  # 形成DataFrame

a    0
b    1
dtype: int64
c    2
d    3
e    4
dtype: int64
f    5
g    6
dtype: int64
==============[1]=============
a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64
==============[2]=============
     0    1    2
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0


In [28]:
# 存在堆叠索引
s4 = pd.concat([s1,s3])
print s4
print s1
print "==============[1]============="
print pd.concat([s1,s4],join='inner')
print "==============[2]============="
print pd.concat ([s1,s4],join='inner',axis=1)
print "==============[3]============="
# 指定连接后必须出现索引['a','b','f','z']
print pd.concat ([s1,s4],join='inner',axis=1,join_axes=[['a','b','f','z']])

a    0
b    1
f    5
g    6
dtype: int64
a    0
b    1
dtype: int64
==============[1]=============
a    0
b    1
a    0
b    1
f    5
g    6
dtype: int64
==============[2]=============
   0  1
a  0  0
b  1  1
==============[3]=============
     0    1
a  0.0  0.0
b  1.0  1.0
f  NaN  5.0
z  NaN  NaN


In [29]:
# 指定连接数据来自哪个Series
print pd.concat([s1,s2,s3],axis=0
                ,keys=['s1','s2','s3'])
pd.concat([s1,s2,s3],axis=1,keys=['s1','s2','s3'])

s1  a    0
    b    1
s2  c    2
    d    3
    e    4
s3  f    5
    g    6
dtype: int64


,s1,s2,s3
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


#### 2. DataFrame的轴连接
1. DataFrame的轴连接 , 逻辑与Series一致.  
  1. 如: 给拼接DataFrame设置keys参数, 指定列/行索引的高层索引, 声明数据来自哪个DtaFrame
  2. DtaFrame指定keys参数外, 还可通过pd.concat(dict{'levelName1':df1,'levelName2':df2})来达到同样效果  
   传入的dict的key当做高层level的索引名
  3. 当不想保留连接的2个DataFrame各自的索引时, 指定参数ignore_index=True, 在新生成的DataFrame上重计索引  
   指定重建索引后, keys参数失效 (因为keys参数是在原有索引的基础上, 在轴连接方向加上高层索引)

In [30]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],columns=['three', 'four'])
print df1
print "==============[1]============="
print df2

   one  two
a    0    1
b    2    3
c    4    5
==============[1]=============
   three  four
a      5     6
c      7     8


In [31]:
# 声明数据来自哪个DtaFrame
print pd.concat([df1,df2],keys=['df1','df2'], axis=1)
pd.concat([df1,df2],keys=['df1','df2'], axis=0)

  df1       df2     
  one two three four
a   0   1   5.0  6.0
b   2   3   NaN  NaN
c   4   5   7.0  8.0


four  one  three  two
df1 a   NaN  0.0    NaN  1.0
    b   NaN  2.0    NaN  3.0
    c   NaN  4.0    NaN  5.0
df2 a   6.0  NaN    5.0  NaN
    c   8.0  NaN    7.0  NaN

In [32]:
# 通过dict指定连接后的高层索引名
pd.concat({'df1':df1,'df2':df2})

four  one  three  two
df1 a   NaN  0.0    NaN  1.0
    b   NaN  2.0    NaN  3.0
    c   NaN  4.0    NaN  5.0
df2 a   6.0  NaN    5.0  NaN
    c   8.0  NaN    7.0  NaN

In [33]:
# 重计索引
pd.concat([df1,df2],keys=['df1','df2'], ignore_index=True)

,four,one,three,two
0,NaN,0.0,NaN,1.0
1,NaN,2.0,NaN,3.0
2,NaN,4.0,NaN,5.0
3,6.0,NaN,5.0,NaN
4,8.0,NaN,7.0,NaN


### 4.4 合并重叠索引的数据
1. Series.combine_first(otherSeries) : 用otherSeries中的数据, 对Series中的数据打补丁  
 即用与otherSeries索引不重叠部分接到调用Series数据的后面

In [34]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64),index=['f', 'e', 'd', 'c', 'b', 'a'])
print a
print b

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64


In [35]:
# Series重叠索引数据替换
print b[:-2]
print a[2:]
print "==============[1]============="
print b[:-2].combine_first(a[2:])
print a[2:].combine_first(b[:-2])

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
==============[1]=============
a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64
a    NaN
b    4.5
c    3.5
d    2.0
e    1.0
f    0.0
dtype: float64


In [36]:
# DataFrame打补丁
df1 = DataFrame({'aa':[1,2,3,4],'bb':[5,6,7,8]},index=list('abcd'))
df2 = DataFrame({'aa':[98,99],'bb':[98,99]},index=list('ab'))
df1

,aa,bb
a,1,5
b,2,6
c,3,7
d,4,8


In [37]:
df2

,aa,bb
a,98,98
b,99,99


In [38]:
df2.combine_first(df1)

,aa,bb
a,98.0,98.0
b,99.0,99.0
c,3.0,7.0
d,4.0,8.0


In [39]:
df1.combine_first(df2)

,aa,bb
a,1,5
b,2,6
c,3,7
d,4,8


### 4.5 轴向旋转
重新排列表格形状的操作, 称为绕轴旋转. stack,unstack为一组, pd.melt与DataFrame.pivot为一组, 后者简便, 可代替前者

#### 1. 重塑层次索引
1. 轴旋转的2个方式  
  1. 列"旋转为"行 : DataFrame.stack (顺时针旋转): 返回Series
  2. 行"旋转为"列 : Series/DataFrame.unstack (逆时针旋转): 返回DataFrame  
2. stack与unstack默认在最内层旋转,传入分层级别的编号/名称, 可对其他级别进行stack/unstack操作  
 即 : DataFrame.stack与数据结构.unstack可以指定轴名称. 且旋转后新生成的索引处于最低层次
3. unstack进行逆时针旋转时, 若发现有的index没有数据, 则会引入缺失值  
4. stack顺时针旋转, 会略处缺失数据  
 参数dropna=False保留缺失数据

In [40]:
df = DataFrame(np.arange(6).reshape(2,3),
               index = pd.Index(['Ohio','Colorado'],name = "state"),
               columns = pd.Index(['one','two','three'], name = 'number')) 
df

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [41]:
result = df.stack()
print result
print df.stack().index

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64
MultiIndex(levels=[[u'Ohio', u'Colorado'], [u'one', u'two', u'three']],
           labels=[[0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2]],
           names=[u'state', u'number'])


In [42]:
print result.unstack()

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5


In [43]:
# 改变旋转的层级
result.unstack("state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [44]:
# unstack引入缺失值
s1 = Series([0,1,2,3],index=list('abcd'))
s2 = Series([4,5,6],index=list('cde'))
obj = pd.concat([s1,s2],keys=['one','two'])
print obj
unstackobj = obj.unstack()
unstackobj

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64


,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [45]:
# stack掠出缺失数据
print unstackobj.stack()
# 不让stack略除数据
unstackobj.stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64


one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [46]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df


side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [47]:
# 指定旋转轴名, 旋转后新生成index处于最低级
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

### 4.6 '长格式'转为'宽格式'
现有一个需求, 使用stack()方法转换s数据称长格式  
 把下述DataFrame转换成有3列的数据: Date和Item,value  
  1. Date: year+quarter组成的日期  
  2. Item: 为'realgdp', 'infl', 'unemp'这三种值  
  3. value: 将Item列中的类型对应的数值写在这里

In [48]:
df = pd.read_csv("examples/macrodata.csv")
df.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [49]:
# 1. 用stack的方法, 把上述DataFrame转换成有3列的数据: Date和Item,value 
#     Date: year+quarter组成的日期
#     Item: 为'realgdp', 'infl', 'unemp'这三种值
#     value: 将Item列中的类型对应的数值写在这里
periods = pd.PeriodIndex(year=df.year, quarter=df.quarter,name='date')
print periods
columns = pd.Index(['realgdp', 'infl', 'unemp'],name='item')
print columns
# 重建列索引
df2 = df.reindex(columns=columns)
df2.head()

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name=u'date', length=203, freq='Q-DEC')
Index([u'realgdp', u'infl', u'unemp'], dtype='object', name=u'item')


item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


In [50]:
# 将PeriodIndex中的季度信息, 转化为季度末的"天"日期
df2.index = periods.to_timestamp('D','end')
print df2.index
df2.head()

DatetimeIndex(['1959-03-31', '1959-06-30', '1959-09-30', '1959-12-31',
               '1960-03-31', '1960-06-30', '1960-09-30', '1960-12-31',
               '1961-03-31', '1961-06-30',
               ...
               '2007-06-30', '2007-09-30', '2007-12-31', '2008-03-31',
               '2008-06-30', '2008-09-30', '2008-12-31', '2009-03-31',
               '2009-06-30', '2009-09-30'],
              dtype='datetime64[ns]', name=u'date', length=203, freq='Q-DEC')


item,realgdp,infl,unemp
date,,,
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6
1960-03-31,2847.699,2.31,5.2


In [51]:
print df2.stack().head()
print "==============[1]=============="
print df2.stack().reset_index().head()
print "==============[2]=============="
longdf = df2.stack().reset_index().rename(columns={0:'value'}) # rename(columns=dict()), 将columnIdx中值为key的转换成value
longdf.head()

date        item   
1959-03-31  realgdp    2710.349
            infl          0.000
            unemp         5.800
1959-06-30  realgdp    2778.801
            infl          2.340
dtype: float64
==============[1]==============
        date     item         0
0 1959-03-31  realgdp  2710.349
1 1959-03-31     infl     0.000
2 1959-03-31    unemp     5.800
3 1959-06-30  realgdp  2778.801
4 1959-06-30     infl     2.340
==============[2]==============


,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340


#### 1. pivot使用
1. 基于以上的长格式, 转换为宽格式. 一个简单的方法, 使用DataFrame.pivot()  
  1. 参数index : 哪列columnIdx作为新生成的index
  2. 参数columns : 将哪列columnIdx的取值展开成新的column
  3. 参数value: 新生成column后, 该列的取值为原先DataFrame的哪一列  
2. 如果忽略value这个参数, 会导致columns指定列中出现的值变成其他column的下层columnIdx  
\[注]: pivot其实是先用set_index(参数columns),再用unstack()

In [52]:
longdf.head(10)

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [53]:
# pivot(index,columns,values) : 将长表转换为宽表
longdf.pivot('date','item','value').head()

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [54]:
# 忽略value, 形成多级columnIdx
longdf['value2'] = Series(np.random.rand(longdf.shape[0]))
longdf.head(10)

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,0.592823
1,1959-03-31,infl,0.000,0.714759
2,1959-03-31,unemp,5.800,0.813380
3,1959-06-30,realgdp,2778.801,0.009456
4,1959-06-30,infl,2.340,0.445995
5,1959-06-30,unemp,5.100,0.458368
6,1959-09-30,realgdp,2775.488,0.066815
7,1959-09-30,infl,2.740,0.339802
8,1959-09-30,unemp,5.300,0.052751
9,1959-12-31,realgdp,2785.204,0.603031


In [55]:
longdf.pivot('date','item').head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  0.714759  0.592823  0.813380
1959-06-30  2.34  2778.801   5.1  0.445995  0.009456  0.458368
1959-09-30  2.74  2775.488   5.3  0.339802  0.066815  0.052751
1959-12-31  0.27  2785.204   5.6  0.903368  0.603031  0.729376
1960-03-31  2.31  2847.699   5.2  0.193329  0.977440  0.498736

### 4.7 宽格式转为长格式

#### 1. pd.melt
1. pd.melt相当于unpivot, 选择原表的一列作为标识列, 其他几列作为观测变量, 让这几个观测变量绕轴旋转成行  
  1. id_vars : 标识符列
  2. value_vars : 需转换为一个记录的某列值的列名
  3. var_name : 由列名转换为一个记录的中的某列的值后, 给这些列取得名字
  4. value_name : 给"value"列取值

In [68]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})
# DataFrame.columns指定列名
# DataFrame.columns.names指定columnIdx的层级名
# df.columns = [list('abc'),list('def')]
# df.columns.names=['l1','l2']
# df.index.names= ['aa']
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [69]:
# 固定A, 旋转BC为row
pd.melt(df,id_vars='A',value_vars=['B','C'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


In [74]:
# 固定A, 旋转BC为row
widedf = pd.melt(df,'A',['B','C'],var_name='parameters',value_name='myvalue')
widedf

,A,parameters,myvalue
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


In [78]:
# pivot进行逆操作
longdf = widedf.pivot('A','parameters','myvalue')
longdf.columns.names=[''] #清除column的层级别名"parameters"
longdf.reset_index()


,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6
